In [2]:
!pip install accelerate

In [3]:
!pip install kaggle

  Using cached kagglehub-0.2.5-py3-none-any.whl.metadata (18 kB)
Using cached kagglehub-0.2.5-py3-none-any.whl (34 kB)


In [ ]:
!pip install kagglehub

In [25]:
import kagglehub

# Download latest version
path = kagglehub.model_download("james113001/bert-classifier-math/transformers/prefinetuned")

print("Path to model files:", path)

100%|██████████| 775M/775M [01:23<00:00, 9.69MB/s] 
Extracting model files...


Path to model files: /Users/kolbeweathington/.cache/kagglehub/models/james113001/bert-classifier-math/transformers/prefinetuned/1


In [21]:
import torch

GemmaConfig={
    "model" : "../inputs/gemma/transformers/1.1-2b-it/1",
    "device"  'cuda' if torch.cuda.is_available() else 'cpu'
    "api_key": "EMPTY", 
    "max_tokens":1000,
    "base_url":f"http://localhost:11434/v1"
    }

In [20]:
ls

autogen.ipynb   bert.ipynb      deepseek.ipynb  gemma.ipynb


In [22]:

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, AutoModelForSequenceClassification


class GemmaModelClient:
    def __init__(self, config, **kwargs):
        print(f"GemmaModelClient config: {config}")
        self.device = config.get("device", "cpu")
        self.model = AutoModelForCausalLM.from_pretrained(config["model"]).to(self.device)
        self.model_name = config["model"]
        self.tokenizer = AutoTokenizer.from_pretrained(config["model"], use_fast=False)
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        

        # params are set by the user and consumed by the user since they are providing a custom model
        # so anything can be done here
        gen_config_params = config.get("params", {})
        self.max_length = gen_config_params.get("max_length", 256)

        print(f"Loaded model {config['model']} to {self.device}")

        

    # def create(self, params):
        if params.get("stream", False) and "messages" in params:
            raise NotImplementedError("Local models do not support streaming.")
        else:
            num_of_responses = params.get("n", 1)

            # can create my own data response class
            # here using SimpleNamespace for simplicity
            # as long as it adheres to the ClientResponseProtocol

            response = SimpleNamespace()

            inputs = self.tokenizer.apply_chat_template(
                params["messages"], return_tensors="pt", add_generation_prompt=True
            ).to(self.device)
            inputs_length = inputs.shape[-1]

            # add inputs_length to max_length
            max_length = self.max_length + inputs_length
            generation_config = GenerationConfig(
                max_length=max_length,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.eos_token_id,
            )

            response.choices = []
            response.model = self.model_name

            for _ in range(num_of_responses):
                outputs = self.model.generate(inputs, generation_config=generation_config)
                # Decode only the newly generated text, excluding the prompt
                text = self.tokenizer.decode(outputs[0, inputs_length:])
                choice = SimpleNamespace()
                choice.message = SimpleNamespace()
                choice.message.content = text
                choice.message.function_call = None
                response.choices.append(choice)

            return response

    # def message_retrieval(self, response):
    #     """Retrieve the messages from the response."""
    #     choices = response.choices
    #     return [choice.message.content for choice in choices]

    # def cost(self, response) -> float:
    #     """Calculate the cost of the response."""
    #     response.cost = 0
    #     return 0

    # @staticmethod
    # def get_usage(response):
        # returns a dict of prompt_tokens, completion_tokens, total_tokens, cost, model
        # if usage needs to be tracked, else None
        return {}

In [24]:
import torch

solver = GemmaModelClient(GemmaConfig)
prompt = "You are an ai with a simple goal. you will read a math problem written in latex. Then reread the problem this time to fully understand what it is asking you to solve. then create a procedure for another ai to follow and solve to get the correct solution."
prompt
# def seperate_into_steps(model,text):
#     try:
#             inputs = model.tokenizerBERT(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#             inputs = {key: val.to(self.device) for key, val in inputs.items()}

#             with torch.no_grad():
#                 outputs = self.modelBERT(**inputs)

#             predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
#             predicted_class = torch.argmax(predictions, dim=-1).item()
#             return self.label_mapping[predicted_class]
#         except Exception as e:
#             predicted_class = random.randint(0,6)
#             return self.label_mapping[predicted_class]

GemmaModelClient config: {'model': '../inputs/gemma/transformers/1.1-2b-it/1', 'cpuapi_key': 'EMPTY', 'max_tokens': 1000, 'base_url': 'http://localhost:11434/v1'}


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.01s/it]


ImportError: 
GemmaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:


    


def split_into_steps(text):
        try:
            sentences = text.split('. ')
            sentences = [s.strip() for s in sentences if s.strip()]
            steps = [f"Step {i+1}: {sentence}" for i, sentence in enumerate(sentences)]
            return steps
        except Exception as e:
            print(f"Error splitting instructions: {e}")
            return text

    def select_random_instructions(self, csv_file, class_label):
        try:
            df = pd.read_csv(csv_file)
            df = df[pd.to_numeric(df['answer'], errors='coerce').notnull()]
            df = df.drop_duplicates(subset=['problem'])
            filtered_df = df[df['Type'] == class_label.lower()]
            selected_rows = filtered_df.groupby('Type').head(2)
            results = [{'question': row['problem'], 'instruction': row['steps'], 'answer': row['answer']} for _, row in selected_rows.iterrows()]
            return results
        except Exception as e:
            print(f"Error selecting random instructions: {e}")
            return [{'question': '...', 'instruction': 'Step 1: ..., Step 2: ...', 'answer': '\\boxed{0-99}'}]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map='auto',
                offload_folder="offload"
            )
            return tokenizer, model

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))